<H1>PTT 文章標題爬蟲</H1>
Member : Lun & Yoga
功能：
<li>爬取指定看板內的文章標題，並整理成.csv/.txt</li>
<ol>
    <li>無推數版</li>
    <li>有推數版</li>
</ol>
</br>
<li>以關鍵字搜尋文章標題</li>
<ol>
    <li>多關鍵字（or）</li>
    <li>多層關鍵字（and）</li>
</ol>
</br>
<li>看指定文章</li>
<ol>
    <li>輸入關鍵字但只顯示同關鍵字最先的一篇</li>
    <li>輸入關鍵字找全部頁面</li>
</ol>

## 匯入套件

In [1]:
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm
from bs4 import BeautifulSoup

### DataFrame 印出完整行列&字寬對齊

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.unicode.ambiguous_as_wide', True)
pd.set_option('display.unicode.east_asian_width', True)

## 輸入

In [10]:
board = input("輸入英文版名: ")
pages = int(input("查詢頁數: "))

輸入英文版名: ios
查詢頁數: 20


## 所有文章標題

### 無推數版本

In [ ]:
def get_agree(pages, r, soup):
    button = soup.select('input')
    agree = button[0]["value"]
    agree_url = 'https://www.ptt.cc' + agree
    url = agree_url
    print(url)

url = f"https://www.ptt.cc/bbs/{board}/index.html"
over_18 = {'cookie': 'over18=1;'}
title = []
bar = tqdm(range(pages))

with open(f"ptt_{board}_title.csv", 'w', encoding='utf-8-sig') as f: 
    for page in bar :
        r = requests.get(url,headers = over_18)
        soup = BeautifulSoup(r.text, "html.parser")
        div_t = soup.find_all('div', class_="title")
        for t in div_t:   
            f.write(t.text)
            
#         dictionary = dict(zip(tl, nl))
#         fp.write(str(dictionary))
        btn = soup.select('div.btn-group > a')
        up_page_href = btn[3]['href']
        next_page_url = 'https://www.ptt.cc' + up_page_href
        url = next_page_url
        #print(f'已搜尋看板 {board} 第 {page+1} 頁')
df = pd.read_csv(f"ptt_{board}_title.csv",header=None,delimiter='\\t',encoding='utf-8-sig')
df.to_csv(f"ptt_{board}_title.csv", encoding="utf_8_sig",header=False, index=False)

# 整理成 txt
csv_name = f"ptt_{board}_title"
csv2txt(csv_name)

### 有推數版本

In [4]:
url = f"https://www.ptt.cc/bbs/{board}/index.html"
over_18 = {'cookie': 'over18=1;'}
title, push = [], []
bar = tqdm(range(pages))

for page in bar :
    bar.set_description(f'第{page}頁')
    r = requests.get(url,headers = over_18)
    soup = BeautifulSoup(r.text, "html.parser")
    div_t = soup.find_all('div', class_="title")
    div_n = soup.find_all('div', class_="nrec")
    for t in div_t:   
        title.append(t.text)
        
    for n in div_n:
        push.append(n.text) if len(n) != 0 else  push.append(0)
        
    btn = soup.select('div.btn-group > a')
    up_page_href = btn[3]['href']
    next_page_url = 'https://www.ptt.cc' + up_page_href
    url = next_page_url
    #print(f'已搜尋看板 {board} 第 {page+1} 頁')

    

dataset = {'標題':title, '推數':push}
dictionary = pd.DataFrame(dataset)
dictionary = dictionary.replace(r'\n+|\t', '', regex=True)
dictionary.to_csv(f"ptt_{board}_title_push.csv", encoding='utf-8-sig', index=False)
print('finish')
print(dictionary)

csv2txt(f"ptt_{board}_title_push")

第19頁: 100%|██████████████████████████████████████████████████████████████████████████| 20/20 [00:12<00:00,  1.54it/s]s]


finish
                                                                     標題 推數
0          [問卦] 韓國飲食 有哪些口味感覺比台灣好吃的?                       6
1                                [問卦] 女生要會嗲一下                       5
2                   Re: [問卦] 台灣醫生護士怎麼不罷工?                       0
3                              [問卦] 中部百億博弈大亨                      14
4        [問卦] 常聽到的火腿腸是等於台灣的什麼食物呢？                       7
5     Re: [問卦] 逢甲宋同學開車根本小屁孩 被打也應該ㄅ                       1
6              [問卦] 屁孩為說服家長買玩具化身推銷師？                       0
7                Re: [問卦] 真的有很多人鬼切三車道過嗎                       1
8                       [問卦] 有没有 牆外小花的八卦??                       0
9              Re: [問卦] 劉備48歲吃18歲的孫小妹也太爽                       1
10          Re: [問卦] 鬼切宋 應該不會再開車上路了吧？                       3
11                    [問卦] 圓的東西都可愛 除了人類？                       1
12   Re: [新聞] 繼海底撈後！陸網紅手搖店爆87家關店潮　                       3
13           Re: [問卦] 高端仔知道高端為什麼會失敗了嗎                       3
14    Re: [問卦] 逢甲宋同學開車根本小屁孩 被打也

NameError: name 'csv2txt' is not defined

### 整理成 TXT

In [ ]:
def clean(raw_df):
    filled_df = raw_df.fillna(0) # 推文 NaN 值補 0
    title = filled_df[1:][0]
    p = filled_df[1:][1]
    t = title.replace(r'\n+|\t', '', regex=True)
    df = pd.concat([t,p], axis=1)
    return df

# 輸出至 ptt_{board}.txt
def csv2txt(csv_name):
    if 'push' not in csv_name:
        df = pd.read_csv(f'{csv_name}.csv', header=None, delimiter='\\t', encoding='utf-8-sig', engine='python')
        
    elif 'push' in csv_name:
        raw_df = pd.read_csv(f'{csv_name}.csv', header=None, encoding='utf-8-sig', engine='python')
        df = clean(raw_df)

    df.to_csv(f'{csv_name}.txt', encoding='utf_8_sig', header=False, index=False)

# 關鍵字搜尋
輸出至 ptt_{board}_{key}.txt

#### 多關鍵字 ( or )，空格隔開

In [7]:
n = 0
keys = list(input("請輸入關鍵字並以空白間隔:").split(" "))
df = pd.read_csv(f"ptt_{board}_title_push.csv", encoding='utf-8-sig')
for key in keys:
    mask = df['標題'].str.contains(key)
    df[mask].to_csv(f'ptt_{board}_{key}.csv', encoding='utf-8-sig', index=False)
    n += len(df[mask])
    #with open(f"ptt_{board}_{key}.csv", 'a', encoding='utf-8-sig') as f:
        #f.write("共 %d 筆" % len(df[mask]))

dataset = pd.DataFrame()
for key in keys:
    data = pd.read_csv(f'ptt_{board}_{key}.csv', index_col=None)
    dataset = pd.concat([dataset, data])
dataset = dataset.drop_duplicates(['標題']) # 刪除重複標題
dataset.to_csv(f"ptt_{board}_{keys}.csv", encoding='utf-8-sig', header=None, index=False)

with open(f"ptt_{board}_{keys}.csv", 'a', encoding='utf-8-sig') as f:
    f.write("共 %d 筆" % n)
    
print(f'搜索關鍵字 {keys} 完成')
print(dataset)

請輸入關鍵字並以空白間隔:有沒有 詐騙
搜索關鍵字 ['有沒有', '詐騙'] 完成
                                            標題  推數
0          [問卦] 有沒有還沒感恩節馴鹿就來的八卦     1
1      [問卦] 有沒有年輕人不會用app訂電影票的卦?    23
2        [問卦] 有沒有早點名已經可以廢除的八卦？     4
3  Re: [問卦] 有沒有年輕人不會用app訂電影票的卦?     0
4          [問卦] 有沒有北京環球影城威震天的八卦     3
5                  [問卦] 有沒有 Rivian 的八卦？     3
6                  [問卦] 有沒有手機貸款詐騙八卦     4
7                  [問卦] 獵豹清理大師有沒有用呀     0
8           [問卦] 有沒有永續飛航燃油(SAF)的八卦     1
9                [問卦] 有沒有青旅混合房的八卦？     5


#### 多層單關鍵字 ( and )

In [8]:
def MultiFilter(result):
    times = input('輸入還需要篩選幾次: ')
    times = eval(times)
    if times > 0 :
        for i in range(times):
            key = input(f'輸入第{i+1}篩選字 : ')
            keys.append(key)
            mask = result['標題'].str.contains(key)
            result = result[mask]
            result.to_csv(f"ptt{board}_{keys}.csv",encoding='utf-8-sig')
            with open(f"ptt{board}_{keys}.csv",'a',encoding='utf-8-sig') as file:
                file.write("\n共%d筆"%len(result))
            print(result)
            print("共%d筆"%len(result))
        
key = input("請輸入關鍵字:")
keys = []
keys.append(key)
df = pd.read_csv(f"ptt_{board}_title_push.csv", encoding='utf-8-sig')
mask = df['標題'].str.contains(key)
result = df[mask]

result.to_csv(f"ptt{board}_{key}.csv",encoding='utf-8-sig')
result.replace(r'\n+|\t', '', regex=True)
with open(f"ptt{board}_{key}.csv",'a',encoding='utf-8-sig') as file:
    file.write("\n共 %d 筆" % len(result))
print(result)
print("共 % d筆" % len(result))
MultiFilter(result)

請輸入關鍵字:有沒有
                                              標題 推數
101          [問卦] 有沒有還沒感恩節馴鹿就來的八卦    1
108      [問卦] 有沒有年輕人不會用app訂電影票的卦?   23
111        [問卦] 有沒有早點名已經可以廢除的八卦？    4
114  Re: [問卦] 有沒有年輕人不會用app訂電影票的卦?    0
142          [問卦] 有沒有北京環球影城威震天的八卦    3
200                  [問卦] 有沒有 Rivian 的八卦？    3
300                  [問卦] 有沒有手機貸款詐騙八卦    4
311                  [問卦] 獵豹清理大師有沒有用呀    0
380           [問卦] 有沒有永續飛航燃油(SAF)的八卦    1
393                [問卦] 有沒有青旅混合房的八卦？    5
共  10筆
輸入還需要篩選幾次: 2
輸入第1篩選字 : 八卦
                                        標題 推數
101    [問卦] 有沒有還沒感恩節馴鹿就來的八卦    1
111  [問卦] 有沒有早點名已經可以廢除的八卦？    4
142    [問卦] 有沒有北京環球影城威震天的八卦    3
200            [問卦] 有沒有 Rivian 的八卦？    3
300            [問卦] 有沒有手機貸款詐騙八卦    4
380     [問卦] 有沒有永續飛航燃油(SAF)的八卦    1
393          [問卦] 有沒有青旅混合房的八卦？    5
共7筆
輸入第2篩選字 : 詐騙
                              標題 推數
300  [問卦] 有沒有手機貸款詐騙八卦    4
共1筆


<h1>看指定文章</h1>

### 輸入關鍵字但只顯示同關鍵字最先的一篇

輸出至 ptt{board}_{target}.txt

In [11]:
target = input("請輸入標題:")
url = f"https://www.ptt.cc/bbs/{board}/index.html"

def GetContent():
    ar = soup.select('div.title > a')
    ar_href = ar[number]['href']
    ar_url = 'https://www.ptt.cc' + ar_href
    aurl = requests.get(ar_url,headers = over_18)
    asoup = BeautifulSoup(aurl.text,"html.parser")
    CONTENT = asoup.find(id='main-container').text
    return file.write(ar_url+CONTENT)
    
task = 'continue'
with open(f"ptt{board}_target_{target}.txt",'w',encoding='utf-8') as file: 
    for page in range(pages):
        if task == 'continue':
            r = requests.get(url,headers=over_18)
            soup = BeautifulSoup(r.text,"html.parser")
            div = soup.find_all('div',class_="title")
            number = -1
            for t in div:
                number += 1
                if target in t.text:
                    print('Find')
                    GetContent()
                    task = 'complete'
            if task == 'continue':
                btn = soup.select('div.btn-group > a')
                up_page_href = btn[3]['href']
                next_page_url = 'https://www.ptt.cc' + up_page_href
                url = next_page_url
                print(f'搜尋第{page+1}頁',end='')
        else:
            print('task ',task)
            break

請輸入標題:手機
搜尋第1頁搜尋第2頁搜尋第3頁Find
Find
task  complete


### 輸入關鍵字找全部頁面

In [12]:
target = input("請輸入標題:")
url = f"https://www.ptt.cc/bbs/{board}/index.html"

def GetContent():
    ar = soup.select('div.title > a')
    ar_href = ar[number]['href']
    ar_url = 'https://www.ptt.cc' + ar_href
    aurl = requests.get(ar_url,headers=over_18)
    asoup = BeautifulSoup(aurl.text,"html.parser")
    CONTENT = asoup.find(id='main-container').text
    file.write(ar_url+CONTENT+"\n")
    return ar_url,CONTENT

with open(f"ptt{board}_{target}_all.txt",'w',encoding='utf-8') as file: 
    for page in range(pages):
        r = requests.get(url,headers=over_18)
        soup = BeautifulSoup(r.text,"html.parser")
        div = soup.find_all('div',class_="title")
        number = -1
        for t in div:
            number += 1
            if target in t.text:
                print('Find')
                GetContent()
        btn = soup.select('div.btn-group > a')
        up_page_href = btn[3]['href']
        next_page_url = 'https://www.ptt.cc' + up_page_href
        url = next_page_url
        print(f'搜尋第{page+1}頁',end='')
        

請輸入標題:手機
搜尋第1頁搜尋第2頁搜尋第3頁Find
Find
搜尋第4頁Find
Find
搜尋第5頁Find
Find
Find
Find
Find
搜尋第6頁Find
Find
搜尋第7頁Find
搜尋第8頁Find
Find
搜尋第9頁Find
Find
搜尋第10頁Find
Find
搜尋第11頁Find
Find
搜尋第12頁Find
Find
搜尋第13頁Find
搜尋第14頁Find
搜尋第15頁搜尋第16頁Find
搜尋第17頁搜尋第18頁搜尋第19頁Find
Find
搜尋第20頁

------------

### 用ptt搜尋功能

In [ ]:
"""
施工中
"""
from selenium import webdriver


new_value = input("關鍵字")
url = f"https://www.ptt.cc/bbs/{board}/index.html"
browser = webdriver.Chrome("chromedriver.exe")
browser.get(url)

html = browser.page_source
soup = bs4.BeautifulSoup(html,"html.parser")
div = soup.find_all('input',value = '')
div

#search = BeautifulSoup()

----------------------------------------------

### selenium click in (慢)

from selenium import webdriver
from selenium.webdriver.common.by import By
import bs4
import time
import pandas as pd
target = input()
url = f"https://www.ptt.cc/bbs/{board}/index.html"
browser = webdriver.Edge("C:\python3.7\edgedriver_win64\msedgedriver.exe")
browser.get(url)
for i in range(pages): 
    try:
        html = browser.page_source
        soup = bs4.BeautifulSoup(html,"html.parser")
        div = soup.find_all('div',class_="title")
        for t in div:
            if target in t.text:
                browser.find_element_by_link_text(target).click()
                print('Find!')
                time.sleep(100)
            else:
                print("no",end = ',')
        try :
            print(i)
            buttom = browser.find_element_by_xpath("/html/body/div[2]/div[1]/div[1]/div[2]/a[2]")
            buttom.click()
        except:
            break
    except :
        break
browser.close()